# Test if we can load trained weights - make modifications to the model class (to save intermediated feature maps) needed for analysis

TLDR: yes it works !

# Load libraries 

In [1]:
%%bash
pip install attrdict
pip install timm
pip install pytorch-lightning==1.4.0

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import matplotlib.pyplot as plt
import os
import tqdm

import seaborn as sns
from torchvision.io import read_image
import torchvision.transforms as T
from torchvision.utils import make_grid
from attrdict import AttrDict
import torch
import yaml
from sklearn.model_selection import StratifiedKFold
import copy
import pickle
# from tqdm import tqdm_notebook

# additional lightning 

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule


# pytorch
import torch
from torch import nn
import torch.nn.functional as F
from timm import create_model


from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

# Resnet

In [4]:
# https://stackoverflow.com/questions/36700404/tensorflow-opening-log-data-written-by-summarywriter

resnet_list = []
for i in range(5):
    print(f"\n Fold: {i}==================================================")
    path = [x for x in os.listdir(f"../input/02-pytorch-lightning-variant/pawnet_lightning_resnet/default/version_{i}/") if x.startswith("events")][0]
    event_acc = EventAccumulator(os.path.join(f"../input/02-pytorch-lightning-variant/pawnet_lightning_resnet/default/version_{i}/",path), size_guidance={'scalars': 0})
    event_acc.Reload()

    scalars = {}
    for tag in event_acc.Tags()['scalars']:
        events = event_acc.Scalars(tag)
        scalars[tag] = [event.value for event in events]
    resnet_list.append(scalars)

In [5]:
resnet_val = 0
resnet_training_curve = []
resnet_validation_curve = []
for f in resnet_list:
    print(f["val_RMSE_loss"][-1])
    resnet_val +=f["val_RMSE_loss"][-1] / 5
print(f"AVG: {resnet_val}")

# Vit

In [7]:
# https://stackoverflow.com/questions/36700404/tensorflow-opening-log-data-written-by-summarywriter

vit_list = []
for i in range(5):
    print(f"\n Fold: {i}==================================================")
    path = [x for x in os.listdir(f"../input/../input/vit-small/pawnet_lightning_vit_tiny_patch16_224/default/version_{i}/") if x.startswith("events")][0]
    event_acc = EventAccumulator(os.path.join(f"../input/vit-small/pawnet_lightning_vit_tiny_patch16_224/default/version_{i}/",path), size_guidance={'scalars': 0})
    event_acc.Reload()

    scalars = {}
    for tag in event_acc.Tags()['scalars']:
        events = event_acc.Scalars(tag)
        scalars[tag] = [event.value for event in events]
    vit_list.append(scalars)

In [8]:
vit_val = 0
for f in vit_list:
    print(f["val_RMSE_loss"][-1])
    vit_val +=f["val_RMSE_loss"][-1] / 5
print(f"AVG: {vit_val}")

# Visualize all

In [12]:
fig,axes = plt.subplots(nrows=2, figsize=(10,7))
fig.suptitle('Logloss Learning Curve - Train')
fig.supxlabel('Epochs')
fig.supylabel('Logloss')

axes[0].set_title("Resnet34")
axes[0].plot(np.arange(len(resnet_list[0]["train_logloss"])),resnet_list[0]["train_logloss"])
axes[0].plot(np.arange(len(resnet_list[1]["train_logloss"])),resnet_list[1]["train_logloss"])
axes[0].plot(np.arange(len(resnet_list[2]["train_logloss"])),resnet_list[2]["train_logloss"])
axes[0].plot(np.arange(len(resnet_list[3]["train_logloss"])),resnet_list[3]["train_logloss"])
axes[0].plot(np.arange(len(resnet_list[4]["train_logloss"])),resnet_list[4]["train_logloss"])
axes[0].grid()

axes[1].set_title("Vit-Tiny-224")
axes[1].plot(np.arange(len(vit_list[0]["train_logloss"])),vit_list[0]["train_logloss"])
axes[1].plot(np.arange(len(vit_list[1]["train_logloss"])),vit_list[1]["train_logloss"])
axes[1].plot(np.arange(len(vit_list[2]["train_logloss"])),vit_list[2]["train_logloss"])
axes[1].plot(np.arange(len(vit_list[3]["train_logloss"])),vit_list[3]["train_logloss"])
axes[1].plot(np.arange(len(vit_list[4]["train_logloss"])),vit_list[4]["train_logloss"])
axes[1].grid()

In [14]:
fig,axes = plt.subplots(nrows=2, figsize=(10,7))
fig.suptitle('RMSE Learning Curve - Validation')
fig.supxlabel('Epochs')
fig.supylabel('RMSE')

axes[0].set_title("Resnet34")
axes[0].plot(np.arange(len(resnet_list[0]["val_RMSE_loss"])),resnet_list[0]["val_RMSE_loss"])
axes[0].plot(np.arange(len(resnet_list[1]["val_RMSE_loss"])),resnet_list[1]["val_RMSE_loss"])
axes[0].plot(np.arange(len(resnet_list[2]["val_RMSE_loss"])),resnet_list[2]["val_RMSE_loss"])
axes[0].plot(np.arange(len(resnet_list[3]["val_RMSE_loss"])),resnet_list[3]["val_RMSE_loss"])
axes[0].plot(np.arange(len(resnet_list[4]["val_RMSE_loss"])),resnet_list[4]["val_RMSE_loss"])
axes[0].grid()

axes[1].set_title("Vit-Tiny-224")
axes[1].plot(np.arange(len(vit_list[0]["val_RMSE_loss"])),vit_list[0]["val_RMSE_loss"])
axes[1].plot(np.arange(len(vit_list[1]["val_RMSE_loss"])),vit_list[1]["val_RMSE_loss"])
axes[1].plot(np.arange(len(vit_list[2]["val_RMSE_loss"])),vit_list[2]["val_RMSE_loss"])
axes[1].plot(np.arange(len(vit_list[3]["val_RMSE_loss"])),vit_list[3]["val_RMSE_loss"])
axes[1].plot(np.arange(len(vit_list[4]["val_RMSE_loss"])),vit_list[4]["val_RMSE_loss"])
axes[1].grid()